![banner.png](banner.png)

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Imports</h2>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, min, max, count
import os

In [7]:
os.environ["HADOOP_HOME"] = "D:\\hadoop"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["HADOOP_HOME"], "bin")

In [2]:
spark = SparkSession.builder \
    .appName("IMDbProject") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

25/03/08 21:48:29 WARN Utils: Your hostname, LAPTOP-AE18MK00 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/08 21:48:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 21:48:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Confirm Spark is running
print(f"Spark Version: {spark.version}")

Spark Version: 3.5.5


In [ ]:
# spark.conf.set("spark.sql.shuffle.partitions", "4")
# spark.conf.set("spark.executor.memory", "2g")

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Load Datasets</h2>

In [4]:
# Paths
IMDB_PATH = '../data/raw/imdb'
TMDB_PATH = '../data/raw/tmdb'
RAW_PARQUET_PATH = '../data/raw_parquet'
PROCESSED_PATH = '../data/processed'

In [8]:
# Test

# Load just ratings file
ratings = spark.read.csv(
    os.path.join(IMDB_PATH, 'title.ratings.tsv'),
    sep='\t',
    header=True,
    inferSchema=True,
    nullValue='\\N'
)

# Quick check the schema (optional)
ratings.printSchema()

# Quick check the data (optional)
ratings.show(5)

# Test writing ratings to Parquet (this is the test)
ratings.write.mode('overwrite').parquet(os.path.join(PROCESSED_PATH, 'ratings.parquet'))

print("✅ Ratings saved to Parquet successfully!")

ConnectionRefusedError: [Errno 111] Connection refused

In [5]:
# Make sure output folders exist
os.makedirs(RAW_PARQUET_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

imdb_files = {
    "basics": "title.basics.tsv",
    "akas": "title.akas.tsv",
    "crew": "title.crew.tsv",
    "episode": "title.episode.tsv",
    "principals": "title.principals.tsv",
    "ratings": "title.ratings.tsv",
    "names": "name.basics.tsv"
}

### Load IMDB data to Spark Dataframes

In [5]:
imdb_dfs = {
    key: spark.read.csv(
        os.path.join(IMDB_PATH, filename),
        sep='\t',
        header=True,
        inferSchema=True,
        nullValue='\\N'
    )
    for key, filename in imdb_files.items()
}

for name, df in imdb_dfs.items():
    print(f"IMDb - {name}:")
    df.show(5)

IMDb - basics:


+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1|              

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Carmencita|  NULL|    NULL|   original|         NULL|              1|
|tt0000001|       2|          Carmencita|    DE|    NULL|       NULL|literal title|              0|
|tt0000001|       3|          Carmencita|    US|    NULL|imdbDisplay|         NULL|              0|
|tt0000001|       4|Carmencita - span...|    HU|    NULL|imdbDisplay|         NULL|              0|
|tt0000001|       5|          Καρμενσίτα|    GR|    NULL|imdbDisplay|         NULL|              0|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
only showing top 5 rows

IMDb - crew:
+---------+---------+---------+
|   tconst|directors|  writers

+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0031458|  tt32857063|        NULL|         NULL|
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
|tt0042889|   tt0989125|        NULL|         NULL|
|tt0043426|   tt0040051|           3|           42|
+---------+------------+------------+-------------+
only showing top 5 rows

IMDb - principals:


+---------+--------+---------+---------------+--------------------+----------+
|   tconst|ordering|   nconst|       category|                 job|characters|
+---------+--------+---------+---------------+--------------------+----------+
|tt0000001|       1|nm1588970|           self|                NULL|  ["Self"]|
|tt0000001|       2|nm0005690|       director|                NULL|      NULL|
|tt0000001|       3|nm0005690|       producer|            producer|      NULL|
|tt0000001|       4|nm0374658|cinematographer|director of photo...|      NULL|
|tt0000002|       1|nm0721526|       director|                NULL|      NULL|
+---------+--------+---------+---------------+--------------------+----------+
only showing top 5 rows

IMDb - ratings:
+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2136|
|tt0000002|          5.5|     289|
|tt0000003|          6.4|    2170|
|tt0000004|          5.3|     185|
|tt

### Save unfiltered IMDB files to Parquet

In [6]:
parquet_path = os.path.join(RAW_PARQUET_PATH, 'ratings.parquet')
print(f"💾 Saving 'ratings' to {parquet_path} ...")
imdb_dfs['ratings'].write.mode('overwrite').parquet(parquet_path)
print(f"✅ Successfully saved 'ratings' to {parquet_path}!")

💾 Saving 'ratings' to ../data/raw_parquet/ratings.parquet ...


✅ Successfully saved 'ratings' to ../data/raw_parquet/ratings.parquet!


In [7]:
parquet_path = os.path.join(RAW_PARQUET_PATH, 'names.parquet')
print(f"💾 Saving 'names' to {parquet_path} ...")
imdb_dfs['names'].write.mode('overwrite').parquet(parquet_path)
print(f"✅ Successfully saved 'names' to {parquet_path}!")

💾 Saving 'names' to ../data/raw_parquet/names.parquet ...


[Stage 22:==========================================================(8 + 0) / 8]

✅ Successfully saved 'names' to ../data/raw_parquet/names.parquet!


In [7]:
print("📂 Loading IMDb dataset: basics")
imdb_df_basics = spark.read.csv(
    os.path.join(IMDB_PATH, "title.basics.tsv"),
    sep='\t',
    header=True,
    inferSchema=True,
    nullValue='\\N'
)
print(f"✅ Loaded 'basics'")

📂 Loading IMDb dataset: basics


[Stage 3:===================================================>       (7 + 1) / 8]

✅ Loaded 'basics'


In [ ]:
parquet_path = os.path.join(RAW_PARQUET_PATH, 'basics.parquet')
print(f"💾 Saving 'basics' to {parquet_path} ...")
imdb_df_basics.write.mode('overwrite').parquet(parquet_path)
print(f"✅ Successfully saved 'basics' to {parquet_path}!")

💾 Saving 'basics' to ../data/raw_parquet/basics.parquet ...


25/03/08 21:32:09 WARN NettyRpcEnv: Ignored message: HeartbeatResponse(false) 8]
25/03/08 21:32:09 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1219)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.

In [6]:
print("📂 Loading IMDb dataset: akas")
imdb_df_akas = spark.read.csv(
    os.path.join(IMDB_PATH, "title.akas.tsv"),
    sep='\t',
    header=True,
    inferSchema=True,
    nullValue='\\N'
)
print(f"✅ Loaded 'akas'")

📂 Loading IMDb dataset: akas


[Stage 1:===================================================>     (18 + 2) / 20]

✅ Loaded 'akas'


In [ ]:
parquet_path = os.path.join(RAW_PARQUET_PATH, 'akas.parquet')
print(f"💾 Saving 'akas' to {parquet_path} ...")
imdb_df_akas.write.mode('overwrite').parquet(parquet_path)
print(f"✅ Successfully saved 'akas' to {parquet_path}!")

In [ ]:
print("📂 Loading IMDb dataset: crew")
imdb_df_crew = spark.read.csv(
    os.path.join(IMDB_PATH, "title.crew.tsv"),
    sep='\t',
    header=True,
    inferSchema=True,
    nullValue='\\N'
)
print(f"✅ Loaded 'crew'")

In [ ]:
parquet_path = os.path.join(RAW_PARQUET_PATH, 'crew.parquet')
print(f"💾 Saving 'crew' to {parquet_path} ...")
imdb_df_crew.write.mode('overwrite').parquet(parquet_path)
print(f"✅ Successfully saved 'crew' to {parquet_path}!")

In [ ]:
print("📂 Loading IMDb dataset: principals")
imdb_df_principals = spark.read.csv(
    os.path.join(IMDB_PATH, "title.principals.tsv"),
    sep='\t',
    header=True,
    inferSchema=True,
    nullValue='\\N'
)
print(f"✅ Loaded 'principals'")

In [ ]:
parquet_path = os.path.join(RAW_PARQUET_PATH, 'principals.parquet')
print(f"💾 Saving 'principals' to {parquet_path} ...")
imdb_df_principals.write.mode('overwrite').parquet(parquet_path)
print(f"✅ Successfully saved 'principals' to {parquet_path}!")

In [ ]:
print("📂 Loading IMDb dataset: episode")
imdb_df_episode = spark.read.csv(
    os.path.join(IMDB_PATH, "title.episode.tsv"),
    sep='\t',
    header=True,
    inferSchema=True,
    nullValue='\\N'
)
print(f"✅ Loaded 'episode'")

In [ ]:
parquet_path = os.path.join(RAW_PARQUET_PATH, 'episode.parquet')
print(f"💾 Saving 'episode' to {parquet_path} ...")
imdb_df_episode.write.mode('overwrite').parquet(parquet_path)
print(f"✅ Successfully saved 'episode' to {parquet_path}!")

### Load TMDB data to Spark Dataframes

In [ ]:
tmdb_path = os.path.join(TMDB_PATH, 'TMDB_movie_dataset_v11.csv')

tmdb_df = spark.read.csv(
    tmdb_path,
    header=True,
    inferSchema=True,
    nullValue='\\N'
)

print("TMDb Data Preview:")
tmdb_df.show(5)

In [7]:
tmdb_path = os.path.join(TMDB_PATH, 'TMDB_movie_dataset_v11.csv')

tmdb_df = spark.read.csv(
    tmdb_path,
    header=True,
    inferSchema=True,
    nullValue='\\N'
)

print("TMDb Data Preview:")
tmdb_df.show(5)

TMDb Data Preview:
+------+---------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|          title|vote_average|vote_count|  status|release_date|   revenue|runtime|adult|       backdrop_path|   budget|            homepage|  imdb_id|original_language| original_title|            overview|          popularity|         poster_path|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+------+---------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+---------------+--------------------+-------

### Save TMDB file to Parquet

In [ ]:
TMDB_PARQUET_PATH = '../data/raw_parquet'
os.makedirs(TMDB_PARQUET_PATH, exist_ok=True)

tmdb_df.write.mode('overwrite').parquet(os.path.join(TMDB_PARQUET_PATH, 'TMDB_movie_dataset_v11.parquet'))
print("Saved TMDb to parquet")

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Exploratory Data Analysis</h2>

### Preliminary EDA

In [8]:
imdb_basics = imdb_dfs['basics']

# Total counts for each dataset
print("Row counts per file:")
for name, df in imdb_dfs.items():
    print(f"{name}: {df.count():,} rows")

# Date range in imdb_basics
date_range = imdb_basics.select(min(col("startYear")).alias("min_year"), max(col("startYear")).alias("max_year"))
date_range.show()

# Top genres
genres_exploded = imdb_basics.withColumn("genre", explode(split(col("genres"), ",")))
top_genres = genres_exploded.groupBy("genre").count().orderBy(col("count").desc())
top_genres.show(10)

Row counts per file:
basics: 11,485,855 rows
akas: 51,496,209 rows
crew: 11,485,855 rows
episode: 8,832,680 rows
principals: 91,154,932 rows
ratings: 1,539,107 rows
names: 14,217,591 rows
+--------+--------+
|min_year|max_year|
+--------+--------+
|    1874|    2031|
+--------+--------+

+-----------+-------+
|      genre|  count|
+-----------+-------+
|      Drama|3237636|
|     Comedy|2236773|
|  Talk-Show|1424798|
|      Short|1228429|
|Documentary|1096234|
|       News|1080456|
|    Romance|1072271|
|     Family| 844672|
| Reality-TV| 643344|
|  Animation| 572468|
+-----------+-------+
only showing top 10 rows



### Loading Using Parquet Files

In [ ]:
# # Load from saved Parquets (raw)
# imdb_dfs = {}
# imdb_files = ["basics", "akas", "crew", "episode", "principals", "ratings", "names"]

# for name in imdb_files:
#     path = os.path.join(RAW_PARQUET_PATH, f'{name}.parquet')
#     imdb_dfs[name] = spark.read.parquet(path)

### Filter to `movies` only

In [7]:
imdb_basics_filtered = imdb_dfs['basics'].filter(col("titleType") == "movie")

# # Save filtered basics immediately to processed folder
# imdb_basics.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'basics.parquet'))

In [8]:
# Filter akas
imdb_akas_filtered = imdb_dfs['akas'] \
    .join(imdb_basics_filtered.select("tconst"), imdb_dfs['akas'].titleId == imdb_basics_filtered.tconst, "inner") \
    .drop("tconst")

# imdb_akas.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'akas.parquet'))

# Filter ratings
imdb_ratings_filtered = imdb_dfs['ratings'] \
    .join(imdb_basics_filtered.select("tconst"), "tconst", "inner")

# imdb_ratings.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'ratings.parquet'))

# Filter principals
imdb_principals_filtered = imdb_dfs['principals'] \
    .join(imdb_basics_filtered.select("tconst"), "tconst", "inner")

# imdb_principals.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'principals.parquet'))

# Filter crew
imdb_crew_filtered = imdb_dfs['crew'] \
    .join(imdb_basics_filtered.select("tconst"), "tconst", "inner")

# imdb_crew.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'crew.parquet'))

# For now, leaving `names` alone since it's a people table not directly tied to titleType.
imdb_names = imdb_dfs['names']
# imdb_dfs['names'].write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'names.parquet'))

# Episodes is usually for TV shows, so we can skip it, or handle if needed later.
imdb_episodes = imdb_dfs['episode']

#### Record Counts

In [ ]:
imdb_basics_filtered.count()
imdb_akas_filtered.count()
imdb_ratings_filtered.count()
imdb_principals_filtered.count()
imdb_crew_filtered.count()

In [ ]:
tmdb_movies.count()

#### Schema Checks

In [ ]:
# Print schemas for IMDb files
for name, df in imdb_dfs.items():
    print(f"Schema for IMDb file: {name}")
    df.printSchema()
    print("="*40)  # Just for cleaner separation

# Print schema for TMDb file
print("Schema for TMDb file:")
tmdb_df.printSchema()

### 1. Highest Rated Movies in 2023

**Balancing Ratings and Number of Votes:**



#### Filter 2023 Data

### 2. Most Popular Actors/Actresses in 2023

### 3. User-Movie Trends

### 4. Metric of a 'Hit Movie'

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Predictive Modelling</h2>